In [ ]:
pwd

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
caminho = 'D:\DeepNLP\basez\movie_titles_metadata.txt'

arq = open(r'D:\DeepNLP\base\movie_titles_metadata_new.txt')

In [ ]:
arq.read()

In [ ]:
arq = open()


In [ ]:
arq.seek(0)

In [ ]:
conversas = open(r'D:\DeepNLP\base\movie_conversations.txt', encoding='utf8').read().split('/n')
type(conversas)
conversas

In [ ]:
print(arq.read())

In [4]:
import pandas as pd
import re

In [ ]:
#df = pd.read_csv(r'D:\DeepNLP\base\movie_titles_metadata.txt',sep=' +++$+++ ')
df = pd.read_csv(r'D:\DeepNLP\base\movie_titles_metadata_new.txt', header=None,  engine= 'python', sep=';')
# correct Separator should be sep=' +++$+++ ' I've got an error, so I need to clean the results
# I've change the original file to use ";" as Separator

In [ ]:
df.head()


**movie_titles_metadata.txt**
	
    - contains information about each movie title
	- fields: 
		- movieID, 
		- movie title,
		- movie year, 
	   	- IMDB rating,
		- no. IMDB votes,
 		- genres in the format ['genre1','genre2',É,'genreN']

In [ ]:

df.columns = ['movieID','movieTitle','movieYear','IMDBRating','IMDBVotes','genres']
#change  columns name folowing description above

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['subURL']= 'N/A'
#this new column will keep subtitle URL to new corpus.
df['found']= 'N/A'
#this new Column will help to control status


In [ ]:
df.dtypes


**Searching for subtitles**
    I'll get subtitles in OpenSubtitles.org and catalog in pandas dataframe
    

In [70]:
import time
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [ ]:
driver = webdriver.Firefox()
driver.get('https://www.opensubtitles.org/')



In [ ]:
pd.set_option('display.max_colwidth',200)
df['subURL']

In [ ]:
# Find subtitles and save them
x = range(len(df))
for i in x :
  
  driver.find_element(By.ID, "search_text").clear()
  driver.implicitly_wait(1)
  driver.find_element(By.ID, "search_text").send_keys(df['movieTitle'][i])
  
  driver.find_element(By.ID, "search_text").send_keys(Keys.ENTER)
  driver.implicitly_wait(6)
  # Check if specific DIV exists to get data instead of list->div class="msg" itemscope="" itemtype="http://schema.org/Movie"
  # Xpath /html/body/div[1]/div[11]
  # CSS Selector = div.msg:nth-child(15)
  element = driver.find_element_by_xpath("/html/body/div[1]/div[11]")
  # Xpath /html/body/div[1]/div[11]
  # CSS Selector = div.msg:nth-child(15) 
  if len(element.text) > 0 : 
     df.loc[i,'found']= 'ok'
  else:
     df.loc[i,'found']= 'nok'
    #not tested from here but can work
     driver.find_element(By.ID, "search_text").clear()
     driver.implicitly_wait(1)
     driver.find_element(By.ID, "search_text").send_keys(df['movieTitle'][i]+' ('+ df['movieYear'][i]+')')
     driver.find_element(By.ID, "search_text").send_keys(Keys.ENTER)
     driver.implicitly_wait(6)
     element = driver.find_element_by_xpath("/html/body/div[1]/div[11]")
     if len(element.text) > 0 : 
       df.loc[i,'found']= 'ok'
     #not tested til here
          
  df.loc[i,'subURL']= driver.current_url
  time.sleep(7)
  print (i) #just to see how many lines was already read
    
driver.quit()

In [ ]:
#save results in CSV file
df.to_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL.csv', encoding='utf-8', sep=';')

**Results and next step**
    #1 - get all records that have in "found" column an "ok" attibute and concatenate at the end of URL "/sort-7/asc-0" to order the results for the most downloaded subtitles then download.
    #2 - get all records that have in "found" column an "nok" attibute and get the correct URLs for the movies, then execute step 1
    #3 - after a visual inspection I've marked some records with "check" atribute, so we need to check manually.

In [ ]:
#I did some changes and a manually inspection on file,and rename it
#everything was marked as "OK" but url had "\sserach\" I change and update manually
#load results to start to work
df_result = pd.read_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 1stProcess.csv',  engine= 'python', sep=';')
#df_result = pd.read_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 2ndProcess.csv',  engine= 'python', sep=';')


In [ ]:
#some 
df_result.head(10)

In [ ]:
df_result.dtypes

In [ ]:

#create new dataframe to work just with "nok" cases
#found correct URL to download subtitles
df_nok=df_result.query('found =="nok"')


In [ ]:
df_nok.dtypes


In [ ]:
df_nok.reset_index(drop=True, inplace=True)
df_nok.head()

In [ ]:
driver = webdriver.Firefox()

In [ ]:
# find and save corret URL to download page
element = ''
for x in range(len(df_nok)) :
    URL = df_nok['subURL'][x]
    driver.get(URL)
    elementBkp = element
    try :
      element = driver.find_element_by_xpath('//*[@id="search_results"]')
      html_content = element.get_attribute('outerHTML')
      soup = BeautifulSoup(html_content, 'html.parser')
    except:
      element = elementBkp
      #just a Detour

    for i in range(len(soup.find_all('a'))) :        
       if soup.find_all('a')[i].get_text().replace('\n',' ').lower().find((df_nok['movieTitle'][x]+' ('+df_nok['movieYear'][x]+')').lower())>= 0 : 
          print(soup.find_all('a')[i],'\n') 
          #print('https://www.opensubtitles.org'+soup.find_all('a')[i].get('href'))
          df_result.loc[df_nok['Unnamed: 0'][x],'found']= 'upd'
          df_result.loc[df_nok['Unnamed: 0'][x],'subURL']= 'https://www.opensubtitles.org'+soup.find_all('a')[i].get('href')  
          break 
            

In [ ]:
df_result.to_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 2ndProcess.csv', encoding='utf-8', sep=';')

**Load results**

I did some changes mannualy just to check if everything is OK.

I'm going to download those subtitles, that I already have URLs, save and clean those files.

In [17]:
df_result = pd.read_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 2ndProcess.csv',  engine= 'python', sep=';')

In [4]:
df_ok=df_result.query('found !="nok"')

In [5]:
df_ok.reset_index(drop=True, inplace=True)
df_ok.tail()

,Unnamed: 0,movieID,movieTitle,movieYear,IMDBRating,IMDBVotes,genres,subURL,found
574,612,m612,watchmen,2009,7.8,135229,"['action', 'crime', 'fantasy', 'mystery', 'sci...",https://www.opensubtitles.org/pt/search/sublan...,ok
575,613,m613,xxx,2002,5.6,53505,"['action', 'adventure', 'crime']",https://www.opensubtitles.org/pt/search/sublan...,ok
576,614,m614,x-men,2000,7.4,122149,"['action', 'sci-fi']",https://www.opensubtitles.org/pt/search/sublan...,ok
577,615,m615,young frankenstein,1974,8.0,57618,"['comedy', 'sci-fi']",https://www.opensubtitles.org/pt/search/sublan...,ok
578,616,m616,zulu dawn,1979,6.4,1911,"['action', 'adventure', 'drama', 'history', 'w...",https://www.opensubtitles.org/pt/search/imdbid...,ok


In [6]:

#create a new column to keep file URL.
df_ok['subDownUrl']= 'N/A'


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
df_ok.dtypes

Unnamed: 0      int64
movieID        object
movieTitle     object
movieYear       int64
IMDBRating    float64
IMDBVotes       int64
genres         object
subURL         object
found          object
subDownUrl     object
dtype: object

In [10]:
import time
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC


In [28]:
#1 concatenar "/sort-7/asc-0" no final da URL
element = ''

c = DesiredCapabilities.FIREFOX
c["pageLoadStrategy"] = "none"

driver = webdriver.Firefox(desired_capabilities=c)
#driver = webdriver.Firefox()
for x in range(len(df_ok)) :
    w = WebDriverWait(driver, 15)
    URL = df_ok['subURL'][x]+'/sort-7/asc-0' # get the most downloaded subtitle.
    print(str(x)+URL )
    driver.get(URL)
    elementBkp = element
    try :
    
     #release the process as soon as locate the element we are looking for.
      w.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/form/table/tbody/tr[2]/td[1]/strong/a')))
      element = driver.find_element_by_xpath('/html/body/div[1]/form/table/tbody/tr[2]/td[1]/strong/a') 
      #get the exactly right Xpath to link.
      html_content = element.get_attribute('outerHTML')
      soup = BeautifulSoup(html_content, 'html.parser')
      print('passou aqui')
    except:
      element = elementBkp
      #just a Detour

    #passo 2 Achar as URLs corretas para trabalhar 

    try:
      df_ok.loc[x,'subDownUrl']= 'https://www.opensubtitles.org'+soup.find('a').get('href')
      df_ok.loc[x,'found']= 'dwl'
      print(soup.find('a').get('href'))
      print('indice - '+ str(x))
    except:
      df_ok.loc[x,'found']= 'err'
      print('Erro Linha'+ str(x))
    
    #2 criar processo de download e gravação dos arquivo
    time.sleep(7)  
driver.close()
df_ok.to_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 3rdProcess.csv', encoding='utf-8', sep=';')

296https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-27625/sort-7/asc-0
/pt/subtitles/3080403/crash-pb
indice - 296
297https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-2058/sort-7/asc-0
passou aqui
/pt/subtitles/3565593/the-crow-pb
indice - 297
298https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-919/sort-7/asc-0
passou aqui
/pt/subtitles/3565593/the-crow-pb
indice - 298
299https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-7447/sort-7/asc-0
passou aqui
/pt/subtitles/128455/cruel-intentions-pb
indice - 299
300https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-54674/sort-7/asc-0
passou aqui
/pt/subtitles/3119136/the-curse-of-the-cat-people-pb
indice - 300
301https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-139/sort-7/asc-0
passou aqui
/pt/subtitles/3642870/dark-city-pb
indice - 301
302https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-2829/sort-7/asc-0
passou aqui
/pt/subtitles

347https://www.opensubtitles.org/pt/search/imdbid-97388/sublanguageid-pob/moviename-friday%20the%2013th%20part%20viii:%20jason%20takes%20manhattan/sort-7/asc-0
passou aqui
/pt/subtitles/69202/jason-goes-to-hell-the-final-friday-pb
indice - 347
348https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-18559/sort-7/asc-0
passou aqui
/pt/subtitles/56610/friday-the-13th-part-viii-jason-takes-manhattan-pb
indice - 348
349https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-1745/sort-7/asc-0
passou aqui
/pt/subtitles/3506353/gandhi-pb
indice - 349
350https://www.opensubtitles.org/pt/search/imdbid-119177/sublanguageid-pob/moviename-gattaca/sort-7/asc-0
passou aqui
/pt/subtitles/3506353/gandhi-pb
indice - 350
351https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-881/sort-7/asc-0
passou aqui
/pt/subtitles/3179208/gattaca-pb
indice - 351
352https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-4983/sort-7/asc-0
passou aqui
/pt/subtitles/4998152/

395https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-6289/sort-7/asc-0
passou aqui
/pt/subtitles/3537370/freddy-vs-jason-pb
indice - 395
396https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-5268/sort-7/asc-0
passou aqui
/pt/subtitles/3374973/kafka-pb
indice - 396
397https://www.opensubtitles.org/pt/search/imdbid-67309/sublanguageid-pob/moviename-klute/sort-7/asc-0
passou aqui
/pt/subtitles/3528796/king-kong-pb
indice - 397
398https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-2160/sort-7/asc-0
passou aqui
/pt/subtitles/5812698/klute-pb
indice - 398
399https://www.opensubtitles.org/pt/search/imdbid-119485/sublanguageid-pob/moviename-kundun/sort-7/asc-0
passou aqui
/pt/subtitles/3119519/kramer-vs-kramer-pb
indice - 399
400https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-1717/sort-7/asc-0
passou aqui
/pt/subtitles/3119802/kundun-pb
indice - 400
401https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-2460/sort-7/a

446https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-1202/sort-7/asc-0
passou aqui
/pt/subtitles/3118008/pearl-harbor-pb
indice - 446
447https://www.opensubtitles.org/pt/search/imdbid-105128/sublanguageid-pob/moviename-pet%20sematary%20ii/sort-7/asc-0
passou aqui
/pt/subtitles/3139534/peggy-sue-got-married-pb
indice - 447
448https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-1607/sort-7/asc-0
passou aqui
/pt/subtitles/97930/pet-sematary-ii-pb
indice - 448
449https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-122/sort-7/asc-0
passou aqui
/pt/subtitles/3482114/philadelphia-pb
indice - 449
450https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-4/sort-7/asc-0
passou aqui
/pt/subtitles/3367159/pitch-black-pb
indice - 450
451https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-1855/sort-7/asc-0
passou aqui
/pt/subtitles/99188/planet-of-the-apes-pb
indice - 451
452https://www.opensubtitles.org/pt/search/sublanguageid-pob

498https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-160/sort-7/asc-0
passou aqui
/pt/subtitles/3107762/south-park-bigger-longer-uncut-pb
indice - 498
499https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-458/sort-7/asc-0
passou aqui
/pt/subtitles/154060/sphere-pb
indice - 499
500https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-205/sort-7/asc-0
passou aqui
/pt/subtitles/3537742/star-wars-episode-iv-a-new-hope-pb
indice - 500
501https://www.opensubtitles.org/pt/search/imdbid-79945/sublanguageid-pob/moviename-star%20trek:%20the%20motion%20picture/sort-7/asc-0
passou aqui
/pt/subtitles/23252/starship-troopers-pb
indice - 501
502https://www.opensubtitles.org/pt/search/imdbid-120202/sublanguageid-pob/moviename-state%20and%20main/sort-7/asc-0
passou aqui
/pt/subtitles/3167125/star-trek-the-motion-picture-pb
indice - 502
503https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-5035/sort-7/asc-0
passou aqui
/pt/subtitles/3253261/state

549https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-3182/sort-7/asc-0
passou aqui
/pt/subtitles/3118037/star-trek-v-the-final-frontier-pb
indice - 549
550https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-2940/sort-7/asc-0
passou aqui
/pt/subtitles/3652261/tron-pb
indice - 550
551https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-1636/sort-7/asc-0
passou aqui
/pt/subtitles/3181319/true-lies-pb
indice - 551
552https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-1625/sort-7/asc-0
passou aqui
/pt/subtitles/3395973/true-romance-pb
indice - 552
553https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-1123/sort-7/asc-0
passou aqui
/pt/subtitles/3113671/12-monkeys-pb
indice - 553
554https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-2483/sort-7/asc-0
passou aqui
/pt/subtitles/3512990/u-turn-pb
indice - 554
555https://www.opensubtitles.org/pt/search/sublanguageid-pob/idmovie-2851/sort-7/asc-0
passou aqui
/p

## One last thing
the steps above does not bring correct URL, so we have to discovery correct link


In [1]:
import time
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC


In [11]:
#pegar a verdadeira url de download
element = ''
c = DesiredCapabilities.FIREFOX

c["pageLoadStrategy"] = "none"

driver = webdriver.Firefox(desired_capabilities=c)

for x in range(len(df_ok)) :
    w = WebDriverWait(driver, 15)
    URL = df_ok['subDownUrl'][x] # get the real subtitle URL.
    print(str(x)+URL )
    driver.get(URL)
    elementBkp = element
    try :    
     #release the process as soon as locate the element we are looking for.
      w.until(EC.presence_of_element_located((By.XPATH, '//*[@id="bt-dwl-bt"]')))
      element = driver.find_element_by_xpath('//*[@id="bt-dwl-bt"]') 
      #get the exactly right Xpath to link.
      html_content = element.get_attribute('outerHTML')
      soup = BeautifulSoup(html_content, 'html.parser')
      print('passou aqui')
    except:
      element = elementBkp

    try:
      df_ok.loc[x,'subDownUrl']= 'https://www.opensubtitles.org'+ soup.find(id="bt-dwl-bt").get('href')
      df_ok.loc[x,'found']= 'file'
      print(soup.find('a').get('href'))
      print('indice - '+ str(x))
    except:
      df_ok.loc[x,'found']= 'err'
      print('Erro Linha'+ str(x))
    
    #2 criar processo de download e gravação dos arquivo
    time.sleep(7)
driver.close()
df_ok.to_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 4thProcess.csv', encoding='utf-8', sep=';')

0https://www.opensubtitles.org/pt/subtitles/4253498/10-things-i-hate-about-you-pb
passou aqui
/pt/subtitleserve/sub/4253498
indice - 0
1https://www.opensubtitles.org/pt/subtitles/3141352/1492-conquest-of-paradise-pb
passou aqui
/pt/subtitleserve/sub/4253498
indice - 1
2https://www.opensubtitles.org/pt/subtitles/138875/15-minutes-pb
passou aqui
/pt/subtitleserve/sub/3141352
indice - 2
3https://www.opensubtitles.org/pt/subtitleserve/sub/138875
passou aqui
/pt/subtitleserve/sub/138875
indice - 3
4https://www.opensubtitles.org/pt/subtitles/84899/48-hrs-pb
passou aqui
/pt/subtitleserve/sub/138875
indice - 4
5https://www.opensubtitles.org/pt/subtitles/4958459/the-fifth-element-pb
passou aqui
/pt/subtitleserve/sub/84899
indice - 5
6https://www.opensubtitles.org/pt/subtitles/3659627/8mm-pb
passou aqui
/pt/subtitleserve/sub/4958459
indice - 6
7https://www.opensubtitles.org/pt/subtitles/44143/a-nightmare-on-elm-street-4-the-dream-master-pb
passou aqui
/pt/subtitleserve/sub/3659627
indice - 7
8ht

66https://www.opensubtitles.org/pt/subtitles/3093823/godzilla-pb
passou aqui
/pt/subtitleserve/sub/3173858
indice - 66
67https://www.opensubtitles.org/pt/subtitles/3177123/galaxy-quest-pb
passou aqui
/pt/subtitleserve/sub/3093823
indice - 67
68https://www.opensubtitles.org/pt/subtitleserve/sub/6825422
passou aqui
/pt/subtitleserve/sub/3177123
indice - 68
69https://www.opensubtitles.org/pt/subtitles/3106841/get-shorty-pb
passou aqui
/pt/subtitleserve/sub/3177123
indice - 69
70https://www.opensubtitles.org/pt/subtitles/3947661/ghost-ship-pb
passou aqui
/pt/subtitleserve/sub/3106841
indice - 70
71https://www.opensubtitles.org/pt/subtitles/5679393/ghost-world-pb
passou aqui
/pt/subtitleserve/sub/3947661
indice - 71
72https://www.opensubtitles.org/pt/subtitles/3110058/the-ghost-and-the-darkness-pb
passou aqui
/pt/subtitleserve/sub/5679393
indice - 72
73https://www.opensubtitles.org/pt/subtitles/3157342/ghostbusters-ii-pb
passou aqui
/pt/subtitleserve/sub/3110058
indice - 73
74https://www.op

132https://www.opensubtitles.org/pt/subtitles/4479792/mighty-morphin-power-rangers-the-movie-pb
passou aqui
/pt/subtitleserve/sub/4124798
indice - 132
133https://www.opensubtitles.org/pt/subtitles/3247883/mobsters-pb
passou aqui
/pt/subtitleserve/sub/4479792
indice - 133
134https://www.opensubtitles.org/pt/subtitleserve/sub/5070423
passou aqui
/pt/subtitleserve/sub/3247883
indice - 134
135https://www.opensubtitles.org/pt/subtitles/59294/mr-smith-goes-to-washington-pb
passou aqui
/pt/subtitleserve/sub/3247883
indice - 135
136https://www.opensubtitles.org/pt/subtitles/5766999/mr-deeds-goes-to-town-pb
passou aqui
/pt/subtitleserve/sub/59294
indice - 136
137https://www.opensubtitles.org/pt/subtitles/3105104/the-mummy-pb
passou aqui
/pt/subtitleserve/sub/5766999
indice - 137
138https://www.opensubtitles.org/pt/subtitles/3634918/mystery-men-pb
passou aqui
/pt/subtitleserve/sub/3105104
indice - 138
139https://www.opensubtitles.org/pt/subtitles/5219130/next-friday-pb
passou aqui
/pt/subtitlese

196https://www.opensubtitles.org/pt/subtitles/3502063/the-rock-pb
passou aqui
/pt/subtitleserve/sub/3268347
indice - 196
197https://www.opensubtitles.org/pt/subtitles/3504711/the-majestic-pb
passou aqui
/pt/subtitleserve/sub/3502063
indice - 197
198https://www.opensubtitles.org/pt/subtitles/3092648/the-birds-pb
passou aqui
/pt/subtitleserve/sub/3504711
indice - 198
199https://www.opensubtitles.org/pt/subtitles/3256100/the-body-snatcher-pb
passou aqui
/pt/subtitleserve/sub/3092648
indice - 199
200https://www.opensubtitles.org/pt/subtitles/4149626/the-crow-salvation-pb
passou aqui
/pt/subtitleserve/sub/3256100
indice - 200
201https://www.opensubtitles.org/pt/subtitles/3452619/the-day-the-earth-stood-still-pb
passou aqui
/pt/subtitleserve/sub/4149626
indice - 201
202https://www.opensubtitles.org/pt/subtitles/4108993/the-lost-boys-pb
passou aqui
/pt/subtitleserve/sub/3452619
indice - 202
203https://www.opensubtitles.org/pt/subtitles/3679290/the-thing-pb
passou aqui
/pt/subtitleserve/sub/41

261https://www.opensubtitles.org/pt/subtitles/3713805/blood-simple-pb
passou aqui
/pt/subtitleserve/sub/3143304
indice - 261
262https://www.opensubtitles.org/pt/subtitles/3280465/blow-pb
passou aqui
/pt/subtitleserve/sub/3713805
indice - 262
263https://www.opensubtitles.org/pt/subtitles/4427733/the-battle-of-algiers-pb
passou aqui
/pt/subtitleserve/sub/3280465
indice - 263
264https://www.opensubtitles.org/pt/subtitles/3094256/body-of-evidence-pb
passou aqui
/pt/subtitleserve/sub/4427733
indice - 264
265https://www.opensubtitles.org/pt/subtitles/3099787/the-bridges-of-madison-county-pb
passou aqui
/pt/subtitleserve/sub/3094256
indice - 265
266https://www.opensubtitles.org/pt/subtitles/6904299/bones-pb
passou aqui
/pt/subtitleserve/sub/3099787
indice - 266
267https://www.opensubtitles.org/pt/subtitles/239909/bound-pb
passou aqui
/pt/subtitleserve/sub/6904299
indice - 267
268https://www.opensubtitles.org/pt/subtitles/3652611/the-bourne-identity-pb
passou aqui
/pt/subtitleserve/sub/239909


327https://www.opensubtitles.org/pt/subtitles/3640751/the-exorcist-pb
passou aqui
/pt/subtitleserve/sub/5196040
indice - 327
328https://www.opensubtitles.org/pt/subtitles/3483224/the-fabulous-baker-boys-pb
passou aqui
/pt/subtitleserve/sub/3640751
indice - 328
329https://www.opensubtitles.org/pt/subtitles/3492224/face-off-pb
passou aqui
/pt/subtitleserve/sub/3483224
indice - 329
330https://www.opensubtitles.org/pt/subtitles/4418988/the-family-man-pb
passou aqui
/pt/subtitleserve/sub/3492224
indice - 330
331https://www.opensubtitles.org/pt/subtitleserve/sub/3108107
passou aqui
/pt/subtitleserve/sub/4418988
indice - 331
332https://www.opensubtitles.org/pt/subtitles/3170303/fantastic-voyage-pb
passou aqui
/pt/subtitleserve/sub/4418988
indice - 332
333https://www.opensubtitles.org/pt/subtitles/3256074/fargo-pb
passou aqui
/pt/subtitleserve/sub/3170303
indice - 333
334https://www.opensubtitles.org/pt/subtitles/3329980/fight-club-pb
passou aqui
/pt/subtitleserve/sub/3256074
indice - 334
335h

391https://www.opensubtitles.org/pt/subtitles/3290057/jurassic-park-iii-pb
passou aqui
/pt/subtitleserve/sub/3377253
indice - 391
392https://www.opensubtitles.org/pt/subtitles/3289906/the-lost-world-jurassic-park-pb
passou aqui
/pt/subtitleserve/sub/3290057
indice - 392
393https://www.opensubtitles.org/pt/subtitles/3290058/jurassic-park-pb
passou aqui
/pt/subtitleserve/sub/3289906
indice - 393
394https://www.opensubtitles.org/pt/subtitles/3537370/freddy-vs-jason-pb
passou aqui
/pt/subtitleserve/sub/3290058
indice - 394
395https://www.opensubtitles.org/pt/subtitles/3374973/kafka-pb
passou aqui
/pt/subtitleserve/sub/3537370
indice - 395
396https://www.opensubtitles.org/pt/subtitles/3528796/king-kong-pb
passou aqui
/pt/subtitleserve/sub/3374973
indice - 396
397https://www.opensubtitles.org/pt/subtitles/5812698/klute-pb
passou aqui
/pt/subtitleserve/sub/3528796
indice - 397
398https://www.opensubtitles.org/pt/subtitles/3119519/kramer-vs-kramer-pb
passou aqui
/pt/subtitleserve/sub/5812698
i

456https://www.opensubtitles.org/pt/subtitles/3396297/the-producers-pb
passou aqui
/pt/subtitleserve/sub/3623786
indice - 456
457https://www.opensubtitles.org/pt/subtitles/3487050/psycho-pb
passou aqui
/pt/subtitleserve/sub/3396297
indice - 457
458https://www.opensubtitles.org/pt/subtitles/3436998/maniac-pb
passou aqui
/pt/subtitleserve/sub/3487050
indice - 458
459https://www.opensubtitles.org/pt/subtitles/3128748/vampyr-pb
passou aqui
/pt/subtitleserve/sub/3436998
indice - 459
460https://www.opensubtitles.org/pt/subtitles/5025945/mystery-of-the-wax-museum-pb
passou aqui
/pt/subtitleserve/sub/3128748
indice - 460
461https://www.opensubtitles.org/pt/subtitleserve/sub/4961258
passou aqui
/pt/subtitleserve/sub/5025945
indice - 461
462https://www.opensubtitles.org/pt/subtitles/3537740/star-wars-episode-vi-return-of-the-jedi-pb
passou aqui
/pt/subtitleserve/sub/5025945
indice - 462
463https://www.opensubtitles.org/pt/subtitles/66597/the-rocky-horror-picture-show-pb
passou aqui
/pt/subtitles

521https://www.opensubtitles.org/pt/subtitles/3670168/the-limey-pb
passou aqui
/pt/subtitleserve/sub/28770
indice - 521
522https://www.opensubtitles.org/pt/subtitles/3622901/the-man-who-wasn-t-there-pb
passou aqui
/pt/subtitleserve/sub/3670168
indice - 522
523https://www.opensubtitles.org/pt/subtitles/3312072/the-relic-pb
passou aqui
/pt/subtitleserve/sub/3622901
indice - 523
524https://www.opensubtitles.org/pt/subtitles/3612715/the-truman-show-pb
passou aqui
/pt/subtitleserve/sub/3312072
indice - 524
525https://www.opensubtitles.org/pt/subtitles/3520474/the-x-files-pb
passou aqui
/pt/subtitleserve/sub/3612715
indice - 525
526https://www.opensubtitles.org/pt/subtitles/4063403/the-cell-pb
passou aqui
/pt/subtitleserve/sub/3520474
indice - 526
527https://www.opensubtitles.org/pt/subtitles/6234103/the-third-man-pb
passou aqui
/pt/subtitleserve/sub/4063403
indice - 527
528https://www.opensubtitles.org/pt/subtitles/44470/the-beach-pb
passou aqui
/pt/subtitleserve/sub/6234103
indice - 528
52

**Post Processing**
After process and finally get all subtile URLs, I check and change some lines manually, now I'll join them

In [18]:

df_ok=pd.read_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 4thProcess.csv',  engine= 'python', sep=';')

In [19]:
df_ok.reset_index(drop=True, inplace=True)


In [20]:
df_full = df_result.merge(df_ok[['movieID','found','subDownUrl']], left_on='movieID', right_on='movieID' , how='left')

In [22]:
#df_result.drop(labels='subDownUrl', axis=1,inplace=True)
#df_full.drop(labels='found_y', axis=1,inplace=True)
df_full.dtypes

Unnamed: 0      int64
movieID        object
movieTitle     object
movieYear       int64
IMDBRating    float64
IMDBVotes       int64
genres         object
subURL         object
found_x        object
subDownUrl     object
dtype: object

In [23]:
df_full.columns = ['index','movieID','movieTitle','movieYear','IMDBRating','IMDBVotes','genres','subURL','found','subDownUrl']

In [28]:
df_full.to_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - Final.csv', encoding='utf-8', sep=';')

In [27]:
df_full.query('found=="nok"')

,index,movieID,movieTitle,movieYear,IMDBRating,IMDBVotes,genres,subURL,found,subDownUrl
48,48,m48,dark angel,1990,5.3,3452,"['action', 'crime', 'drama', 'horror', 'sci-fi', 'thriller']",https://www.opensubtitles.org/pt/search2/sublanguageid-pob/moviename-dark+angel,nok,NaN
82,82,m82,"happy birthday, wanda june",1971,5.8,144,"['comedy', 'drama']","https://www.opensubtitles.org/pt/search2/sublanguageid-pob/moviename-happy+birthday,+wanda+june",nok,NaN
115,115,m115,labor of love,1998,4.5,45,[],https://www.opensubtitles.org/pt/search2/sublanguageid-pob/moviename-labor+of+love,nok,NaN
138,138,m138,my mother dreams the satan's disciples in new york,1998,6.7,70,['short'],CURTAyork,nok,NaN
141,141,m141,mumford,1999,6.8,6333,"['comedy', 'drama']",https://www.opensubtitles.org/pt/search2/sublanguageid-pob/moviename-mumford,nok,NaN
144,144,m144,napoleon,1995,5.5,486,"['family', 'adventure']",https://www.opensubtitles.org/pt/search2/sublanguageid-pob/moviename-napoleon,nok,NaN
156,156,m156,panther,1995,5.9,1129,['drama'],https://www.opensubtitles.org/pt/search2/sublanguageid-pob/moviename-panther,nok,NaN
160,160,m160,plastic man,1999,6.7,29,['drama'],https://www.opensubtitles.org/pt/search2/sublanguageid-pob/moviename-plastic+man,nok,NaN
181,181,m181,simone,2010,8.3,9,"['short', 'drama', 'horror', 'thriller']",https://www.opensubtitles.org/pt/search2/sublanguageid-pob/moviename-simone,nok,NaN
183,183,m183,slash,2002,4.2,899,"['comedy', 'horror', 'thriller']",https://www.opensubtitles.org/pt/search2/sublanguageid-pob/moviename-slash,nok,NaN


In [15]:
 pd.set_option("display.max_colwidth", 300)

# Download and catalog files



In [16]:
df_ok['subDownUrl']



0      https://www.opensubtitles.org/pt/subtitleserve/sub/4253498
1      https://www.opensubtitles.org/pt/subtitleserve/sub/4253498
2      https://www.opensubtitles.org/pt/subtitleserve/sub/3141352
3       https://www.opensubtitles.org/pt/subtitleserve/sub/138875
4       https://www.opensubtitles.org/pt/subtitleserve/sub/138875
                                  ...                            
574    https://www.opensubtitles.org/pt/subtitleserve/sub/3402194
575    https://www.opensubtitles.org/pt/subtitleserve/sub/3643190
576     https://www.opensubtitles.org/pt/subtitleserve/sub/124647
577    https://www.opensubtitles.org/pt/subtitleserve/sub/3448782
578    https://www.opensubtitles.org/pt/subtitleserve/sub/3135425
Name: subDownUrl, Length: 579, dtype: object

In [2]:
#pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9686 sha256=c2c9c5c02a0e79324ebe28eff7022ce1a961b5f178b3d707f77a04e9b1ef631d
  Stored in directory: c:\users\ts00074\appdata\local\pip\cache\wheels\a1\b6\7c\0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [3]:
import wget




In [12]:
print('Beginning file download with wget module')

url = 'https://www.opensubtitles.org/pt/subtitleserve/sub/3141352'

file = wget.download(url, out=r'D:\DeepNLP\base\sub')
print(file)

Beginning file download with wget module
D:\DeepNLP\base\sub/1492.conquest.of.paradise.(1992).pob.1cd.(3141352) (1).zip


In [13]:
print(file)


D:\DeepNLP\base\sub/1492.conquest.of.paradise.(1992).pob.1cd.(3141352) (1).zip
